## Danish multilingual Analysis

In this notebook we will look at the errors that our model performs in zero-shot mode.

We will use a model trained on OLID

In [1]:
%load_ext autoreload
%autoreload 2
import os
from datetime import datetime
import fire
import torch
import pandas as pd
from torchtext import data
import torch.nn as nn
from transformers import (
    AdamW, BertForSequenceClassification, BertTokenizer,
    get_constant_schedule_with_warmup
)

from offenseval.nn import (
    Tokenizer,
    train, evaluate, train_cycle, save_model, load_model, evaluate_dataset
)
from offenseval.datasets import datasets

pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 300

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model, TEXT = load_model("../../models/bert_cased.distant.pt", device)
model.eval();

Create fields and some other boilerplate

In [14]:
from offenseval.nn.evaluation import evaluate
from offenseval.nn import EvaluationReport
from tqdm.auto import tqdm

dataset_path = datasets["danish"]["dev"]

report = evaluate_dataset(model, TEXT, dataset_path)
report

Loading dataset...
Building iterators


Acc: 88.18% Macro F1 0.675 (P 0.417 - N 0.934)

Get the predictions

In [16]:
import pandas as pd


df_da = pd.read_table(dataset_path, index_col=0)

df_da["label"] = df_da["subtask_a"] == 'OFF' 
df_da["prob"] = report.probas.numpy()
df_da["pred"] = df_da["prob"] > 0.5

df_da

,tweet,subtask_a,label,prob,pred
id,,,,,
2308,"##15,29 CM REN DANSK MAND.",NOT,False,0.000450,False
2083,Er der vods et sted?,NOT,False,0.000007,False
263,Og??????,NOT,False,0.000008,False
3266,Meget overraskende at Japan ikke er på listen!,NOT,False,0.000008,False
1871,jeg overvejer næsten at invitere dig på aftensmad.,NOT,False,0.000007,False
...,...,...,...,...,...
347,"Jeg studerer selv i England, i en af de dyreste byer i landet - i forhold til Danmark så ligner det LORT. Hjemløse og folk tabt af systemet over alt, skrald over det hele, alle mine venner er ved at drukne i studie lån og meget misundelige på min SU, biler på den forkerte side a vejen... Jeg kun...",OFF,True,0.000056,False
1395,bare hun huske at betale skat og ik kommer hjem når hun bliver syg og nasser på der danske sundhedsvæsen,NOT,False,0.002999,False
1217,"nogen folk har bare ondt i røven.. men jeg forstår godt du gør det, kunne jeg måske også finde på ud i fremtiden",NOT,False,0.002728,False


## Get the translations

Now, let's check out what our model predicts when using the translations of those exact texts to English. 

First, append the translations to the dataframe

In [17]:
import html

translations = pd.read_json("../../data/translations/Danish/offenseval-da-training-v1-en.json")

for id, row in translations.iterrows():
    translations.loc[id, "en"] = html.unescape(row["en"])
    
translations.set_index("id", inplace=True)

df_da = pd.merge(df_da, translations, how="left", left_index=True, right_index=True)

df_da[:10]

,tweet,subtask_a,label,prob,pred,en
id,,,,,,
2308,"##15,29 CM REN DANSK MAND.",NOT,False,0.000450,False,## 15.29 CM PURE DANISH MAN.
2083,Er der vods et sted?,NOT,False,0.000007,False,Is there a place somewhere?
263,Og??????,NOT,False,0.000008,False,And??????
3266,Meget overraskende at Japan ikke er på listen!,NOT,False,0.000008,False,Very surprising that Japan is not on the list!
1871,jeg overvejer næsten at invitere dig på aftensmad.,NOT,False,0.000007,False,I'm almost considering inviting you to dinner.
181,Brænd hele lortet ned og start forfra.,OFF,True,0.000028,False,Burn the whole shit down and start over.
1,Satans danskjävlar!,OFF,True,0.000613,False,Satan's devil!
2840,Thank you :),NOT,False,0.000007,False,Thank you :)
1190,Dansk propaganda hade funkar bättre om ni hade skrivit lite bättre fejk-svenska. Protip: URL,NOT,False,0.009048,False,Danish propaganda would have worked better if you had written a little better fake-Swedish. Protip: URL


## Predicting translations

Now, predict the translation. We need to create examples in a different way to take the "en" field for translation

In [22]:
from offenseval.nn.fields import create_bert_fields
"""
We need different fields!
"""
ID, SUBTASK_A, TEXT = create_bert_fields(TEXT=TEXT)
translated_fields = {
    "id": ('id', ID),
    "en": ('text', TEXT),
    "subtask_a": ('subtask_a', SUBTASK_A),
}


examples = [{
    **{"id": id},
    **t
} for id, t in df_da.iterrows()]

examples = [data.Example.fromdict(ex, fields=translated_fields) for ex in examples]

translated_dataset = data.Dataset(examples, translated_fields.values())

SUBTASK_A.build_vocab(translated_dataset)

assert SUBTASK_A.vocab.itos == ["NOT", "OFF"]


In [30]:
from offenseval.nn.evaluation import evaluate

translation_it = data.Iterator(
    translated_dataset, batch_size=1, device=device,
    shuffle=False
)

trans_report = evaluate(model, tqdm(translation_it))


df_da["prob_en"] = trans_report.probas.numpy()
df_da["pred_en"] = df_da["prob_en"] > 0.5

trans_report

Acc: 88.01% Macro F1 0.753 (P 0.575 - N 0.930)

Let's check the results in English

In [32]:
df_da["pred_prom"] = (df_da["prob"] + df_da["prob_en"])/2 > 0.5


## Error Analysis

Let's check out the errors

In [33]:
errors = df_da[df_da["label"] != df_da["pred"]]

false_neg = errors[errors["label"]]
false_pos = errors[~errors["label"]]


print(f"There are {len(errors)} errors (out of {len(df_da)} instances)")
print(f"{len(false_neg)} are false negatives and {len(false_pos)} are false positives")

There are 70 errors (out of 592 instances)
52 are false negatives and 18 are false positives


In [34]:
false_pos.sort_values("prob", ascending=False)

,tweet,subtask_a,label,prob,pred,en,prob_en,pred_en,pred_prom
id,,,,,,,,,
32,Bamses julerejse var dope.,NOT,False,0.999994,True,Teddy's Christmas trip was dope.,0.999993,True,True
2597,"Det er fandme noget **Fallout: Nakskov** type shit, det der",NOT,False,0.999994,True,"It's fuckin 'something ** Fallout: Nakskov ** type of shit, that thing",0.999994,True,True
2538,"Den er ikke engang slut endnu. Lad os nu lige se situationen an, ikke?",NOT,False,0.999994,True,"It's not even over yet. Now let's just look at the situation, right?",0.000007,False,True
1081,My god! Feel your pain too min gut. Har de her to sataner og de skider i et væk. Især den grå. Den sorte har det noget nasty langt hår ved exit og har gentagne gange måtte køre Rodalon over gulvet efter hun har taget en røvtur over hele stuegulvet for at komme af med sit tilbageblevne halløj hun...,NOT,False,0.999993,True,My god! Feel your pain too bad. Have these two satans and they shit in one away. Especially the gray one. The black man has some nasty long hair at the exit and has repeatedly had to run Rodalon across the floor after taking an ass ride all over the living room floor to get rid of her leftover p...,0.999994,True,True
1340,Satan er det dig? &#x200B;,NOT,False,0.999992,True,Satan is that you?,0.999992,True,True
3342,"""Kan vi ikke spare de gifteksperter vi har siddende i døgnvagt, og bruge pengene på bedre ting?"" ""Det skulle man tro, Hr. Løkke. Men danskerne er nogle idioter, der spiser deres stueplanter.""",NOT,False,0.999991,True,"""Can't we save the marriage experts we have sitting around the clock and spend the money on better things?"" ""You should think so, Mr. Loke. But the Danes are some idiots who eat their houseplants.""",0.999994,True,True
196,"så længe der er mænd som villigt betaler 5000 for et knald, det er ikke pigerne som er dumme ?",NOT,False,0.999981,True,"as long as there are men willing to pay 5000 for a bang, it's not the girls who are stupid?",0.999994,True,True
846,Så vis dog et billedet af den psykopat .,NOT,False,0.999694,True,Then show a picture of the psychopath.,0.999724,True,True
1274,Læs historien . Og tag ned ved grænsen og hils alle kriminelle udlænninge velkommen og piv ikke den dag du ikke mere kan komme på sygehuset uden at betale,NOT,False,0.999642,True,Read the story. And go down the border and welcome all the criminal aliens and don't pivot the day you can no longer get to the hospital without paying,0.000007,False,False


## False Negative 

In [35]:
false_neg.sort_values("prob", ascending=True)

,tweet,subtask_a,label,prob,pred,en,prob_en,pred_en,pred_prom
id,,,,,,,,,
2533,"Vi gider sgu kun have skåne halland og blekinge, resten kan de sgu beholde.",OFF,True,0.000007,False,"We only wish to have nice palms and bleaches, the rest they can keep.",0.000007,False,False
2704,Danske javle!!,OFF,True,0.000007,False,Danish javelin !!,0.000009,False,False
486,Møgunger.,OFF,True,0.000007,False,Brats.,0.000007,False,False
3220,"Skideflot. Et stk. upvote til Dem, mon signor.",OFF,True,0.000008,False,"Fucking Beautiful. One piece. upvote to you, mon signor.",0.999994,True,True
3438,Det er fandme et stærkt album! Tak for grinet :),OFF,True,0.000008,False,It's a strong album! Thanks for the laugh :),0.000007,False,False
617,Norge^har^Pusur...,OFF,True,0.000009,False,Norway has ^ ^ Pusur ...,0.000008,False,False
3595,Nä nu jävlar!,OFF,True,0.000009,False,Now fucking!,0.999994,True,True
1306,"Igen en ubegavet kvinde , som får taletid",OFF,True,0.000009,False,"Again, a gifted woman who gets talking time",0.000208,False,False
3232,Analgafler,OFF,True,0.000009,False,Anal Forks,0.000009,False,False


In [36]:
from offenseval.nn.evaluation import predict_sentence

def predict(sentence):
    return predict_sentence(model, TEXT, sentence)

predict("Brat!")

8.764372978475876e-06

In [37]:
predict("Analgafler")

9.477135790803004e-06

In [38]:
predict("Hold dig væk fra vores ø, du dansker lort.")

4.4550452003022656e-05

Why this is different from the previous prediction?

In [39]:
df_da.loc[858].tweet

'Det her er vitterligt lort. 0/10.'

In [40]:
predict(df_da.loc[858].tweet)

3.62635291821789e-05

In [41]:
predict(df_da.loc[858].en)

0.9999936819076538

In [42]:
predict(df_da.loc[1408].en)

4.231172351865098e-05

In [43]:
predict(df_da.loc[1408].tweet)

7.441757043125108e-05

## Aciertos

In [44]:
df_da.columns

Index(['tweet', 'subtask_a', 'label', 'prob', 'pred', 'en', 'prob_en',
       'pred_en', 'pred_prom'],
      dtype='object')

In [45]:
correct = df_da[df_da["label"] == df_da["pred"]]

true_pos = correct[correct["label"]]
true_neg = correct[~correct["label"]]

In [46]:
true_pos

,tweet,subtask_a,label,prob,pred,en,prob_en,pred_en,pred_prom
id,,,,,,,,,
2504,"Holy fuck, fik lige en orgasme takket være dette billede.",OFF,True,0.999994,True,"Holy fuck, just got an orgasm thanks to this picture.",0.999994,True,True
387,Shit man! Skal kattene med på ferie? Det er nok en god idé at snakke med kattepasserne om den potentielle lortesituation! God arbejdslyst!,OFF,True,0.999994,True,Shit man! Are the cats going on holiday? It's probably a good idea to talk to cat cats about the potential shit situation! Good work!,0.999994,True,True
11,"Det var da en omgang sludder, der ville noget. Men Yah! død over svensken.",OFF,True,0.990884,True,It was then a game of gossip that would matter. But Yah! death over the Swede.,0.000013,False,False
432,Fuck nogle møgdyr det er jo ren mafia bare for at de kan støtte de åndssvage bådflygtninge,OFF,True,0.999994,True,Fuck some beasts it's pure mafia just so they can support the weak boat refugees,0.999994,True,True
685,"Fucking højrøvet tåbe. Det var da USA'S største fejl at vælge det fjols som president, hold kæft en tegneserie figur",OFF,True,0.999994,True,"Fucking loud ass fool. It was then the US's biggest mistake to pick that fool as president, shut up a cartoon character",0.999994,True,True
2739,"Fuckit, har upvoted dummere i dag.",OFF,True,0.999993,True,"Fuckit, have upvoted fools today.",0.999994,True,True
2736,"Jeg ville fanme være godt knotten, hvis jeg skulle sidde på en satans bænk til et møde. Hvad fanden er det for noget!? Så ville jeg hellere stå op.",OFF,True,0.989317,True,"If I were to sit on a satan's bench for a meeting, I'd be damn good. What the hell is that !? Then I'd rather get up.",0.999994,True,True
58,Fucking smatso af en kælling,OFF,True,0.999994,True,Fucking smatso of a bitch,0.999994,True,True
1943,JA! FUCKING JA!,OFF,True,0.999994,True,YES! FUCKING YES!,0.999994,True,True



We can observe that the transfer learning only occurs because of shared vocabulary...

## Tweets correctly classified in English but not in Danish 

In [47]:
df_da[df_da["pred_en"] & df_da["label"] & ~df_da["pred"]]

,tweet,subtask_a,label,prob,pred,en,prob_en,pred_en,pred_prom
id,,,,,,,,,
181,Brænd hele lortet ned og start forfra.,OFF,True,0.000028,False,Burn the whole shit down and start over.,0.999994,True,True
1,Satans danskjävlar!,OFF,True,0.000613,False,Satan's devil!,0.999994,True,True
634,aldrig hørt om lortet,OFF,True,0.000014,False,never heard of the shit,0.999920,True,False
458,"Så justitsministeren, som jeg formoder har en hovedrolle inden for bekæmpelse af stofmisbrug, har ikke den fjerneste anelse om hvor farlige de enkelte stoffer er? De er skøre, de svenskere.",OFF,True,0.000098,False,"So the Minister of Justice, who I suppose has a leading role in the fight against substance abuse, has no idea how dangerous the individual drugs are? They are crazy, the Swedes.",0.999994,True,True
1538,"Det er så typisk danskere det her.. Typisk dansk sommer: ""det regner for meget, det er en dårlig sommer, Jeg rejser fandme sydpå hvor det er dejligt varmt"" 2018 - rekord sommer! ""det er for varmt, der er for tørt, Jeg må ikke grille, Jeg kan ikke spille computer, min røv sveder osv. "" Den h...",OFF,True,0.000282,False,"It is so typical Danes this .. Typical Danish summer: ""it is raining too much, it is a bad summer, I travel to the south where it is nice warm"" 2018 - record summer! ""it's too hot, it's too dry, I can't grill, I can't play computer, my ass sweats, etc."" This summer is great! Yes it's hot, Yes it...",0.963533,True,False
1028,Hvorfor er svensken så humorforladt og skindød? Og lad da så for helvede Skåne komme hjem.,OFF,True,0.000053,False,Why is the Swede so humorous and skinned? And then let the hell Skåne come home.,0.999994,True,True
265,Ingenting er mer gøy enn å gi svensken en skikkelig smekk på rumpen!,OFF,True,0.000014,False,Nothing is more fun than giving the Swede a good taste on the butt!,0.999993,True,True
273,Jeg vil aldrig mere klage over en lorte sommer.. Bare giv mig regn..,OFF,True,0.000079,False,I will never again complain about a shitty summer .. Just give me rain ..,0.999994,True,True
1010,"Paraply betyder regnskærm på fransk. Regnvejr som undskyldning for at aflyse? Der må være en anden grund. Måske de horder af antifa og andet afskum, der vil slå Trump ihjel.",OFF,True,0.000088,False,Umbrella means rain shield in French. Rainfall as an excuse to cancel? There must be another reason. Maybe the hordes of antifa and other scum that will kill Trump.,0.999994,True,True
